In [ ]:
import numpy as np

bbox_worldcord = [18, 34, 101, 0, 0, 70] # [x,y,z, roll, pitch, yaw]
ego_vehicle_worldcord = [50,60,70,0,0,35] # [x,y,z, roll, pitch, yaw]

def bbox_to_ego(bbox_worldcord:list =[0,0,0,0,0,0], ego_vehicle_worldcord:list =[0,0,0,0,0,0]):
    # bbox_worldcord and ego_vehicle_worldcord are in format [x,y,z, roll, pitch, yaw] (degrees)
    
    def rotation_matrix(roll, pitch, yaw):
        roll = roll * np.pi / 180
        pitch = pitch * np.pi / 180
        yaw = yaw * np.pi / 180

        Rx = np.array([[1, 0, 0],
                       [0, np.cos(roll), -np.sin(roll)],
                       [0, np.sin(roll), np.cos(roll)]])
        
        Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                       [0, 1, 0],
                       [-np.sin(pitch), 0, np.cos(pitch)]])
        
        Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                       [np.sin(yaw), np.cos(yaw), 0],
                       [0, 0, 1]])
        
        return Rz @ Ry @ Rx 
    Re = rotation_matrix(ego_vehicle_worldcord[3], ego_vehicle_worldcord[4], ego_vehicle_worldcord[5])

    Rw = rotation_matrix(bbox_worldcord[3], bbox_worldcord[4], bbox_worldcord[5])
    
    Rbe = Re.T @ Rw
    
    bbox_ego_coords = Re.T @ (np.array(bbox_worldcord[0:3]) - np.array(ego_vehicle_worldcord[0:3]))
    bbox_ego_rotation = np.array([
        np.arctan2(Rbe[2,1], Rbe[2,2]),
        np.arcsin(-Rbe[2,0]),
        np.arctan2(Rbe[1,0], Rbe[0,0])]
    )

    bbox_ego_rotation = bbox_ego_rotation * 180 / np.pi

    return bbox_ego_coords, bbox_ego_rotation

In [17]:
new_coords, new_rotation = bbox_to_ego(bbox_worldcord, ego_vehicle_worldcord)
print("New Coordinates in Ego Frame:", new_coords)
print("New Rotation in Ego Frame (roll, pitch, yaw):", new_rotation)

New Coordinates in Ego Frame: [-41.12585276  -2.94350719  31.        ]
New Rotation in Ego Frame (roll, pitch, yaw): [ 0. -0. 35.]


In [20]:
import h5py
with h5py.File('/home/npopkov/repos/IR2025/data/251119_eight_lidar_10s/db/lidar_data.h5', 'r') as f:
    print(f.keys())

<KeysViewHDF5 ['_carla_ego_vehicle_lidar_right']>
